In [6]:
from bs4 import BeautifulSoup
import requests
import re
from sklearn.base import TransformerMixin
from sklearn.pipeline import make_pipeline, make_union
from sklearn.tree import DecisionTreeClassifier

In [15]:
#doors = requests.get('http://rosettacode.org/wiki/100_doors')

doors = BeautifulSoup(requests.get('http://rosettacode.org/wiki/100_doors').content, 'html.parser')

In [16]:
doors.prettify

<bound method BeautifulSoup.prettify of <!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<title>100 doors - Rosetta Code</title>
<meta charset="utf-8"/>
<meta content="MediaWiki 1.20.3" name="generator"/>
<script type="text/javascript">
//<![CDATA[
try{if (!window.CloudFlare) {var CloudFlare=[{verbose:0,p:0,byc:0,owlid:"cf",bag2:1,mirage2:0,oracle:0,paths:{cloudflare:"/cdn-cgi/nexp/dok3v=1613a3a185/"},atok:"4dbe6bad5f4f1ec3b67fcb79033771d7",petok:"fd015d7a02edf02c30affbb373a37828ba4b7b29-1443812034-1800",zone:"rosettacode.org",rocket:"0",apps:{"ga_key":{"ua":"UA-1219419-1","ga_bs":"2"}}}];!function(a,b){a=document.createElement("script"),b=document.getElementsByTagName("script")[0],a.async=!0,a.src="//ajax.cloudflare.com/cdn-cgi/nexp/dok3v=e9627cd26a/cloudflare.min.js",b.parentNode.insertBefore(a,b)}()}}catch(e){};
//]]>
</script>
<link href="/mw/index.php?title=100_doors&amp;action=edit" rel="alternate" title="Edit" type="application/x-wiki"/>
<link href="/mw/ind

In [18]:
langs = ['c', 'csharp', 'lisp', 'clojure', 'haskell', 
         'java', 'javascript', 'ocaml', 'perl', 'php', 
         'python', 'ruby', 'scala', 'scheme']

# lang_dict = {lang : '' for lang in langs}
lang_X = []
lang_y = []

for string in doors.find_all('pre'):
    for lang in langs:
        if re.search(r'"'+ lang + ' hi', str(string)):
            # lang_dict[lang] = lang_dict[lang] + string.get_text()
            lang_X.append(string)
            lang_y.append(lang)
#print(lang_dict)

In [63]:
for lang in lang_dict:
    print(lang, " - - ", lang_dict[lang], end='\n\n')

javascript  - -   var doors=[];for(var i=0;i<100;i++) doors[i]=false;             //create doorsfor(var i=1;i<=100;i++) for(var i2=i-1,g;i2<100;i2+=i)  doors[i2]=!doors[i2];      //toggle doorsfor(var i=1;i<=100;i++)      //read doors console.log("Door %d is %s",i,doors[i-1]?"open":"closed") for (var door = 1; door <= 100; door++) {  var sqrt = Math.sqrt(door);  if (sqrt === (sqrt | 0)) {    console.log("Door %d is open", door);  }}Array.apply(null, { length: 100 })  .map(function(v, i) { return i + 1; })    .forEach(function(door) {       var sqrt = Math.sqrt(door);        if (sqrt === (sqrt | 0)) {        console.log("Door %d is open", door);      }     });for(var door=1;i<10/*Math.sqrt(100)*/;i++){ console.log("Door %d is open",i*i);} Array.apply(null, { length: 100 })  .map((v, i) => i + 1)    .forEach(door => {       var sqrt = Math.sqrt(door);        if (sqrt === (sqrt | 0)) {        console.log("Door %d is open", door);      }     });// Array comprehension style[ for (i of Array

In [9]:
class FunctionFeaturizer(TransformerMixin):
    def __init__(self, *featurizers):
        self.featurizers = featurizers
        
    def fit(self, X, y=None):
        '''Returns self after transformations'''
        return self
    
    def transform(self, X):
        '''Given a list of original data, return a list of feature vectors'''
        feature_vectors = []
        for x in X:
            feature_vector = [f(x) for f in self.featurizers]
            feature_vectors.append(feature_vector)
        
        return np.array(feature_vectors)

In [11]:
features = FunctionFeaturizer(len)

In [20]:
pipe = make_pipeline(features, DecisionTreeClassifier())
pipe.fit(lang_X, lang_y)
pipe.score(lang_X, lang_y)

NameError: name 'np' is not defined